# Ragas evaluation of smolagents

In [2]:
import os

import torch
from sentence_transformers import SentenceTransformer
from smolagents import LiteLLMModel, ToolCallingAgent

from travel_agent.qdrant import client as qdrant_client
from travel_agent.retrieval.smolagents.tool import GetExistingAvailableRubricsTool, TravelReviewQueryTool

MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0"
model_id = f"ollama_chat/{MODEL_NAME}"


llm = LiteLLMModel(
    model_id=model_id,
    api_base=os.getenv("OLLAMA_URL", "http://127.0.0.1:11434"),
    num_ctx=8192,  # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

test_response = llm([{"role": "user", "content": [{"text": "Алло, это Кизару у телефона?", "type": "text"}]}]).content
print(test_response)
assert test_response

Здравствуйте! Это не Кизару, но я здесь, чтобы помочь.Вы кто-то из мира видеоигр или аниме? Как могу быть полезен?


In [3]:
embed_model_name = "intfloat/multilingual-e5-base"
collection_name = "moskva_intfloat_multilingual_e5_base"


get_available_rubrics_tool = GetExistingAvailableRubricsTool()
review_search_tool = TravelReviewQueryTool(embed_model_name, qdrant_client, collection_name)


agent = ToolCallingAgent(
    tools=[review_search_tool, get_available_rubrics_tool], model=llm, max_steps=10, verbosity_level=2
)

2025-04-27 16:16:17.793 | DEBUG    | travel_agent.qdrant.client:create_client:12 - Connecting to Qdrant at https://21a1c8c4-0d63-4176-9930-ee7848dafd2f.eu-central-1-0.aws.cloud.qdrant.io
2025-04-27 16:16:18.232 | SUCCESS  | travel_agent.qdrant.client:create_client:22 - connected: title='qdrant - vector search engine' version='1.14.0' commit='3617a0111fc8590c4adcc6e88882b63ca4dda9e7'
2025-04-27 16:16:21.571 | INFO     | travel_agent.retrieval.smolagents.tool:__init__:81 - Using device: mps


In [3]:
# Sanity check
agent.run("Что ты думаешь о римской империи?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Что ты думаешь о римской империи?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
"name": "travel_review_query",                                                                                     
"arguments": {                                                                                                     
    "query": "Римская империя отзывы историков",                                                                   
    "min_rating": null,                                                                                            
    "address": "",                                                                                                 
    "rubrics": ""                                                                                                  
}                                                                                                                  
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'Римская империя отзывы историков', 'min_rating': │
│ None, 'address': '', 'rubrics': ''}                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Найденные отзывы о местах:

=== Отзыв на Музей Московских стрельцов Стрелецкие палаты ===
Адрес: Москва, Лаврушинский переулок, 17с1
Рейтинг: 5.0
Категории: |'Музей', 'Выставочный центр']
Текст: Приятный, отзывчивый персонал. Интересная экспозиция. Приятно, что часть экспозиции  интерактивная, можно 
прочувствовать эпоху. Рекомендую к посещению и взрослым, и детям (сын 5ти лет был в восторге) 

=== Отзыв на ВДНХ ===
Адрес: Москва, проспект Мира, 119В
Рейтинг: 5.0
Категории: |'Парк культуры и отдыха']
Текст: Интересно, познавательно, увлекательно,   история, неофит, чисто, востребованно. Если и есть пожелания  так 
это не сбавлять темпы и расширять охват! Молодцы! 

=== Отзыв на Иоанно-Предтеченский Ставропигиальный женский монастырь ===
Адрес: Москва, Малый Ивановский переулок, 2/4с5
Рейтинг: 5.0
Категории: |'Монастырь', 'Православный храм']
Текст: Впечатление просто супер великолепная экскурсия вела матушка часа два очень интересно рассказывала мы 
получили массу положительных эмоций очень советуем посетить это историческое место воодушевиться древними 
реликвиями 

=== Отзыв на Центральный музей Вооружённых Сил Российской Федерации ===
Адрес: Москва, улица Советской Армии, 2, стр. 1
Рейтинг: 5.0
Категории: |'Музей']
Текст: Очень интересный музей, много различной техники, а исторические залы это просто великолепно. Очень 
доброжелательные смотрители музея, пока мы рассматривали различные экспонаты они рассказывали удивительные 
исторические факты. Всем советую посетить данный музей, потому что это будет очень интересным и познавательным 
времяпрепровождением  

=== Отзыв на Музей криптографии ===
Адрес: Москва, Ботаническая улица, 25с4
Рейтинг: 4.0
Категории: |'Музей']
Текст: Хорошее место)  Плюсы: Современное оформление и ремонт Много исторических экспонатов, грамотно составлен 
план осмотра.  Из минусов: Экскурсоводы явно разного уровня подготовки. Первый раз попался идеальный, второй раз  
неповезло. Довольно мало сувенирки (придирка).  На посещение закладывайте 3 часа, иначе не успеете толком 
посмотреть.

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
  "name": "final_answer",                                                                                          
  "arguments": {"answer": "Отзывы о Римской империи напрямую найти сложно. Однако, отклики на исторические музеи   
Москвы отражают общее восхищение и уважение к истории, что аналогично восприятию римского периода. Основные моменты
из этих отзывов: 1) Интересная экспозиция и интерактивность (Музей Московских стрельцов). 2) Воодушевление         
историческими памятниками (Иоанно-Предтеченский Ставропигиальный женский монастырь, Центральный музей Вооружённых  
Сил Российской Федерации). Эти отклики демонстрируют глубокий интерес к истории и культурному наследию, что вполне 
применимо к оценке Римской империи."}                                                                              
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Отзывы о Римской империи напрямую найти сложно.        │
│ Однако, отклики на исторические музеи Москвы отражают общее восхищение и уважение к истории, что аналогично     │
│ восприятию римского периода. Основные моменты из этих отзывов: 1) Интересная экспозиция и интерактивность       │
│ (Музей Московских стрельцов). 2) Воодушевление историческими памятниками (Иоанно-Предтеченский Ставропигиальный │
│ женский монастырь, Центральный музей Вооружённых Сил Российской Федерации). Эти отклики демонстрируют глубокий  │
│ интерес к истории и культурному наследию, что вполне применимо к оценке Римской империи.'}                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Отзывы о Римской империи напрямую найти сложно. Однако, отклики на исторические музеи Москвы отражают
общее восхищение и уважение к истории, что аналогично восприятию римского периода. Основные моменты из этих 
отзывов: 1) Интересная экспозиция и интерактивность (Музей Московских стрельцов). 2) Воодушевление историческими 
памятниками (Иоанно-Предтеченский Ставропигиальный женский монастырь, Центральный музей Вооружённых Сил Российской 
Федерации). Эти отклики демонстрируют глубокий интерес к истории и культурному наследию, что вполне применимо к 
оценке Римской империи.

[Step 2: Duration 18.14 seconds| Input tokens: 4,505 | Output tokens: 230]

'Отзывы о Римской империи напрямую найти сложно. Однако, отклики на исторические музеи Москвы отражают общее восхищение и уважение к истории, что аналогично восприятию римского периода. Основные моменты из этих отзывов: 1) Интересная экспозиция и интерактивность (Музей Московских стрельцов). 2) Воодушевление историческими памятниками (Иоанно-Предтеченский Ставропигиальный женский монастырь, Центральный музей Вооружённых Сил Российской Федерации). Эти отклики демонстрируют глубокий интерес к истории и культурному наследию, что вполне применимо к оценке Римской империи.'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
embedder = SentenceTransformer(embed_model_name, device=device)

In [6]:
# headline_extractor = HeadlinesExtractor(llm=generator_llm, max_num=20)
# headline_splitter = HeadlineSplitter(max_tokens=1500)
# keyphrase_extractor = KeyphrasesExtractor(llm=generator_llm)

# transforms = [headline_extractor, headline_splitter, keyphrase_extractor]

# apply_transforms(kg, transforms=transforms)